In [1]:
import matplotlib
%matplotlib tk
#%matplotlib inline
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

#
from utils.track.track import Track

Autosaving every 180 seconds


In [17]:
root_dir = '/media/cat/4TBSSD/dan/cohort2/cluster_slp_files/'
root_dir = '/media/cat/4TBSSD/dan/mutual_info/early/'
root_dir = '/media/cat/4TBSSD/dan/mutual_info'

period = 'final'

root_dir = os.path.join(root_dir, period)

# use glob to get all *.slp files in the directory
import glob
files = glob.glob(os.path.join(root_dir,'*.npy'))
files.sort()

#
print ("# files: ", len(files))

# convert files from .slp to .npy using the track class
if False:
    plt.figure()
    ctr=0
    for file in files[:10]:
        temp = np.load(file, allow_pickle=True)
        print (temp.shape)

        #
        ax=plt.subplot(3,4,ctr+1)
        ax.set_xlim(0,900)
        ax.set_ylim(0,700)
        for a in range(6):
            #temp2 = np.nanmean(temp[:,a],axis=1)
            temp2 = temp[:,a]

            plt.plot(temp2[:,0], temp2[:,1], 
                    linewidth=2)
            
        plt.title(str(ctr   ))
        ctr+=1



# files:  8


In [19]:
def draw_track(track, 
               ax,
               names):
    
    #
    ax.set_xlim(0,900)
    ax.set_ylim(0,700)
    for a in range(6):
        #temp2 = np.nanmean(temp[:,a],axis=1)
        temp2 = track[:,a]

        plt.plot(temp2[:,0], temp2[:,1], 
                 linewidth=2,
                 label=names[a])
    
    # force ax to respect aspect ratio
    ax.set_aspect('equal', 'box')

#
def compute_1d_mi(track):
    ts1 = track[:,id1,0]
    ts2 = track[:,id2,0]
    nbins1 = 100
    nbins2 = 100

    range1 = [0,900]
    range2 = [0,900]
    
    # Create a 2D histogram
    hist2d, x_edges, y_edges = np.histogram2d(ts1, ts2, bins=(nbins1, nbins2), 
                                            range=[range1, range2])

    # Calculate the marginal probabilities
    px = hist2d.sum(axis=1) / hist2d.sum()
    py = hist2d.sum(axis=0) / hist2d.sum()

    mi = 0
    for i in range(len(px)):
        for j in range(len(py)):
            if hist2d[i, j] > 0:
                mi += hist2d[i, j] * np.log(hist2d[i, j] / (px[i] * py[j]))

    # Normalize the mutual information (optional)
    mi /= np.log(2)

    #
    return mi, hist2d

#
def compute_2d_mi(track):
    from scipy.stats import multivariate_normal
    
    #
    ts1 = track[:,id1]
    ts2 = track[:,id2]

    # find nans indexes
    idx1 = np.where(np.isnan(ts1[:,0])==True)[0]
    idx2 = np.where(np.isnan(ts2[:,0])==True)[0]

    # compute the union of the two
    idx = np.union1d(idx1, idx2)

    # remove all idx from ts1 and ts2
    ts1 = np.delete(ts1, idx, 0)
    ts2 = np.delete(ts2, idx, 0)
    if ts1.shape[0]<10:
        #print ("no data left in ts1")
        return 0

    #check to see if any nans left in ts1
    idx1 = np.where(np.isnan(ts1[:,0])==True)[0]
    idx2 = np.where(np.isnan(ts2[:,0])==True)[0]
    if len(idx1)>0 or len(idx2)>0:
        print ("ERROR: nans left in ts1 or ts2")
        print (idx1, idx2)
        return None

    #
    kde_ts1 = multivariate_normal(np.mean(ts1,axis=0), np.cov(ts1.T))
    kde_ts2 = multivariate_normal(np.mean(ts2,axis=0), np.cov(ts2.T))

    # Evaluate the PDFs at the data points
    pdf_ts1 = kde_ts1.pdf(ts1)
    pdf_ts2 = kde_ts2.pdf(ts2)

    #
    mi = -np.sum(pdf_ts1 * np.log(pdf_ts1 / pdf_ts2))

    #
    return mi

###############################################################
import matplotlib.gridspec as gridspec
names = ['female','male','pup1','pup2','pup3','pup4']

# compute mutual information between the tracks
idx_vids = np.arange(8)
use_1d = True

#
plt.figure(figsize=(10,10))
gs1 = gridspec.GridSpec(3, 6)

# 
ctrv = 0
for v in idx_vids:

    #
    track = np.load(files[v], allow_pickle=True)

    # draw the track first
    ax=plt.subplot(gs1[(v//3), (v%3)*2])
    draw_track(track, ax, names)
    
    #
    if v==0:
        plt.legend(fontsize=6)
        plt.ylabel("animal tracks", fontsize=10)
    
    #
    ax.set_xticks([])
    ax.set_yticks([])
    # 

    plt.title("time: "+ os.path.split(files[v])[1][11:16].replace("_",":"))

    # collapse to just centroid
    #track = np.nanmean(track,axis=2)
    track = track

    #
    mi_array = np.zeros((6,6))+np.nan
    for id1 in range(6):
        for id2 in range(6):
            if id1 == id2:
                continue
            
            #
            if use_1d:
                mi, hist2d = compute_1d_mi(track)
            else:
                mi = compute_2d_mi(track)

            #
            mi_array[id1,id2] = mi

    #
    ax=plt.subplot(gs1[(v//3), (v%3)*2+1])

    plt.imshow(mi_array,
               #aspect='auto',
               )
    
    # change ticks to names
    plt.xticks(np.arange(6), names, fontsize=6)
    plt.yticks(np.arange(6), names, fontsize=6)

    # show colorbar, make ticks small
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=6)
    cbar.ax.set_ylabel('MI', fontsize=6)


    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
    
    #
    ctrv+=1
#
plt.suptitle("dev period: "+period + "\n "+os.path.split(files[0])[1], fontsize=10)

#
plt.show()



In [20]:
plt.figure()

plt.imshow(hist2d)

plt.show()